In [1]:
import shutil
import os
import pandas as pd
""".mer 파일에서 데이터 가공 (차종별 평균)"""
# 파일 경로 (Windows 경로를 올바르게 처리)
folder_path = r"E:\network_version2.9.0"

interval_value = 60
from_time = 1800
# 시뮬레이션 횟수
siml_count = 1
mer_count = 0
mer_name = "network09_mer"

destination_dir = os.path.join(folder_path, mer_name) # 파일명_mer 수정해야함
os.makedirs(destination_dir, exist_ok=True)  # 이미 존재하면 생성하지 않음

mer_list = [file for file in os.listdir(folder_path) if file.endswith(".mer")]
print("총 .mer 파일 수 : ", len(mer_list))

for start in range(0, len(mer_list), siml_count):
    batch_files = mer_list[start:start+siml_count]
    df_list = []
    for file in batch_files:
        mer_path = os.path.join(destination_dir, file)
        print("mer_path : ", mer_path)
        # 파일 읽기
        with open(mer_path, "r", encoding="utf-8", errors="ignore") as file:
            lines = file.readlines()
        # 데이터가 시작하는 인덱스 찾기
        data_start_idx = None

        for j, line in enumerate(lines):
            if "Measurem." in line:  # 컬럼명이 포함된 행 찾기
                data_start_idx = j
                break

        # 데이터프레임 생성
        if data_start_idx is not None:
            # 컬럼명 추출
            columns = lines[data_start_idx].strip().split(";")

            # 데이터 부분 추출 및 가공
            data_lines = lines[data_start_idx + 1:]  # 컬럼명 제외, 데이터 부분
            data = [line.strip().split(";") for line in data_lines if line.strip()]

            # 데이터프레임 생성
            df = pd.DataFrame(data, columns=columns)
            # 컬럼명을 정리 (공백 제거)
            df.columns = df.columns.str.strip()

            # 변환할 숫자형 컬럼 리스트
            numeric_columns = ["Measurem.", "t(Entry)", "t(Exit)", "VehNo", "Vehicle type",
                               "Line", "v[km/h]", "b[m/s2]", "Occ", "Pers", "tQueue", "VehLength[m]"]

            # 각 컬럼을 숫자로 변환
            for col in numeric_columns:
                if col in df.columns:  # 컬럼이 존재하는지 확인
                    df[col] = pd.to_numeric(df[col], errors="coerce")  # 변환 실패 시 NaN 처리

            df["time"] = df[["t(Entry)", "t(Exit)"]].max(axis=1)
            # 시간대별 범위 적용
            current_time = from_time
            max_time = df["time"].max()

            while current_time <= max_time:
                next_time = current_time + interval_value
                df.loc[df["time"].between(current_time, next_time, inclusive="right"), "time"] = next_time
                df.loc[df["time"] == 1800, "time"] = 1860
                current_time = next_time

            display(df)

            df = df.groupby(["Vehicle type", "Measurem.", "time"])[["b[m/s2]", "Occ", "tQueue", "VehLength[m]", "v[km/h]"]].agg("mean").reset_index().round(2)
            df_list.append(df)

    merged_df = pd.concat(df_list, ignore_index=True)
    # 같은 Vehicle type, Measurem., time  기준으로 평균
    avg_df = merged_df.groupby(["Vehicle type", "Measurem.", "time"])[["b[m/s2]", "Occ", "tQueue", "VehLength[m]", "v[km/h]"]].mean().reset_index().round(2)

    merout_path =os.path.join(destination_dir,"vehicle_output")
    os.makedirs(merout_path, exist_ok=True)  # 이미 존재하면 생성하지 않음

    last_file = xlsx_files = sorted([f for f in os.listdir(merout_path) if f.endswith(".xlsx")], key=lambda x: int(x.split("_")[0]) if x.split("_")[0].isdigit() else 0)

    if not xlsx_files:
        last_file = 1
    else:
        last_file = int(xlsx_files[-1].split("_")[0])+1
    excel_path = os.path.join(merout_path,f"{last_file}_vehicle_output.xlsx")
    avg_df.to_excel(excel_path, index=False)
    print("생성파일 : ", excel_path)
    mer_count += 1

    if mer_count >= len(mer_list)/siml_count:
        print("시뮬레이션 분석이 완료되었습니다.")
        break


총 .mer 파일 수 :  36
mer_path :  E:\network_version2.9.0\network09_mer\network09_수정_001.mer


,Measurem.,t(Entry),t(Exit),VehNo,Vehicle type,Line,v[km/h],b[m/s2],Occ,Pers,tQueue,VehLength[m],,time
0,10034,1800.01,-1.00,853,100,0,105.0,0.00,0.09,1,0.0,4.61,,1860.0
1,10107,-1.00,1800.10,707,100,0,89.3,0.00,0.10,1,0.0,4.61,,1860.0
2,10113,1800.07,-1.00,735,100,0,110.7,0.00,0.03,1,0.0,4.76,,1860.0
3,10134,1800.09,-1.00,698,100,0,101.9,0.43,0.01,1,0.0,4.01,,1860.0
4,10190,-1.00,1800.01,604,100,0,106.3,0.00,0.01,1,0.0,3.75,,1860.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1002159,20078,5399.99,-1.00,2574,100,0,102.6,0.00,0.01,1,0.0,4.61,,5400.0
1002160,20120,-1.00,5399.93,2503,100,0,97.3,0.30,0.03,1,0.0,4.64,,5400.0
1002161,20240,5399.94,-1.00,2316,100,0,103.7,-0.17,0.06,1,0.0,4.21,,5400.0
1002162,20267,-1.00,5399.91,2242,100,0,103.1,0.25,0.01,1,0.0,4.61,,5400.0


mer_path :  E:\network_version2.9.0\network09_mer\network09_수정_002.mer


,Measurem.,t(Entry),t(Exit),VehNo,Vehicle type,Line,v[km/h],b[m/s2],Occ,Pers,tQueue,VehLength[m],,time
0,10034,1800.01,-1.00,853,100,0,105.0,0.00,0.09,1,0.0,4.61,,1860.0
1,10107,-1.00,1800.10,707,100,0,89.3,0.00,0.10,1,0.0,4.61,,1860.0
2,10113,1800.07,-1.00,735,100,0,110.7,0.00,0.03,1,0.0,4.76,,1860.0
3,10134,1800.09,-1.00,698,100,0,101.9,0.43,0.01,1,0.0,4.01,,1860.0
4,10190,-1.00,1800.01,604,100,0,106.3,0.00,0.01,1,0.0,3.75,,1860.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1002123,20240,5399.94,-1.00,2316,100,0,103.7,-0.19,0.06,1,0.0,4.21,,5400.0
1002124,20267,-1.00,5399.91,2242,100,0,103.1,0.25,0.01,1,0.0,4.61,,5400.0
1002125,20269,5399.98,-1.00,2258,100,0,109.6,0.00,0.02,1,0.0,4.01,,5400.0
1002126,30166,-1.00,5399.97,2441,100,0,106.7,0.00,0.07,1,0.0,4.21,,5400.0


mer_path :  E:\network_version2.9.0\network09_mer\network09_수정_003.mer


...

,Measurem.,t(Entry),t(Exit),VehNo,Vehicle type,Line,v[km/h],b[m/s2],Occ,Pers,tQueue,VehLength[m],,time
0,10005,1800.04,-1.00,894,630,0,102.9,0.00,0.06,1,0.0,6.67,,1860.0
1,10045,1800.01,-1.00,831,100,0,104.3,-0.25,0.09,1,0.0,4.64,,1860.0
2,10070,1800.05,-1.00,793,100,0,105.2,-0.25,0.05,1,0.0,4.64,,1860.0
3,10086,-1.00,1800.04,762,100,0,105.8,0.00,0.04,1,0.0,4.61,,1860.0
4,10124,-1.00,1800.02,721,100,0,115.3,-0.27,0.02,1,0.0,3.75,,1860.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995869,30091,5399.90,-1.00,2531,630,0,101.3,0.00,0.10,1,0.0,6.67,,5400.0
995870,30097,-1.00,5399.98,2520,100,0,101.1,0.00,0.08,1,0.0,4.21,,5400.0
995871,30124,5399.92,-1.00,2475,100,0,105.0,-0.25,0.08,1,0.0,4.64,,5400.0
995872,30222,5399.97,-1.00,2312,100,0,103.9,0.00,0.03,1,0.0,4.61,,5400.0


mer_path :  E:\network_version2.9.0\network09_mer\network09_수정_006.mer


,Measurem.,t(Entry),t(Exit),VehNo,Vehicle type,Line,v[km/h],b[m/s2],Occ,Pers,tQueue,VehLength[m],,time
0,10005,1800.04,-1.00,894,630,0,102.9,0.00,0.06,1,0.0,6.67,,1860.0
1,10045,1800.01,-1.00,831,100,0,104.3,-0.25,0.09,1,0.0,4.64,,1860.0
2,10070,1800.05,-1.00,793,100,0,105.2,-0.25,0.05,1,0.0,4.64,,1860.0
3,10086,-1.00,1800.04,762,100,0,105.8,0.00,0.04,1,0.0,4.61,,1860.0
4,10124,-1.00,1800.02,721,100,0,115.3,-0.27,0.02,1,0.0,3.75,,1860.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995891,30091,5399.90,-1.00,2531,630,0,101.3,0.00,0.10,1,0.0,6.67,,5400.0
995892,30097,-1.00,5399.98,2520,100,0,101.1,0.00,0.08,1,0.0,4.21,,5400.0
995893,30124,5399.92,-1.00,2475,100,0,105.0,-0.25,0.08,1,0.0,4.64,,5400.0
995894,30222,5399.97,-1.00,2312,100,0,103.9,0.00,0.03,1,0.0,4.61,,5400.0


mer_path :  E:\network_version2.9.0\network09_mer\network09_수정_007.mer


KeyboardInterrupt: 